

# 1. Formulação do problema

### Variáveis de Decisão:
- **ti**: Tempo de pouso do avião i (variável contínua);


- **xij**: Variável binária que vale 1 se o avião i pousar antes do avião j, 0 caso contrário;


- **ei**: Adiantamento do tempo ideal de pouso para o avião i (ei ≥ 0);


- **di**: Atraso do tempo ideal de pouso para o avião i (di ≥ 0).

### Parâmetros:
- **Ri**: Tempo de sua detecção pelo radar;


- **Ei**: Tempo inicial de pouso;


- **Ti**: Tempo ideal para o pouso;


- **Li**: Tempo final que o avião i pode pousar;


### Função Objetivo:
Minimizar a penalidade total de pousos fora do tempo ideal:

   ### **Minimizar ∑ (gi * ei + hi * di)**

Onde:
- **gi** e **hi** são as penalidades por pousar antes ou depois do tempo ideal, respectivamente.

### Restrições:

#### **Restrições de Separação:**
O intervalo entre os pousos deve ser suficiente para garantir segurança e garantir uma sequência válida de aterrissagens:

- **tj ≥ ti + sij − M(1 − xij), ∀i,j, i≠j**  

    Se **xij = 1**, então **tj ≥ ti + sij**, garantindo a separação mínima entre os pousos.  


    Se **xij = 0**, a restrição é relaxada devido ao termo **M(1 − xij)**.

- **ti ≥ tj + sji − Mxij, ∀i,j, i≠j**  
            Similar à restrição anterior, mas garantindo que a relação entre **ti** e **tj** seja consistente dependendo do valor de    **xij**.

#### **Restrições de Sequenciamento:**
Garante que para cada par de aviões **(i, j)**, um deve pousar antes do outro, evitando ciclos:

- **xij + xji = 1, ∀i,j, i≠j**  
        Isso assegura que se o avião **i** pousar antes de **j**, então **xij = 1** e **xji = 0**, caso contrário, **xij = 0** e **xji = 1**.

#### **Cálculo de Adiantamento e Atraso:**

- **ei ≥ Ti − ti ∀i**;


- **di ≥ ti − Ti ∀i**;


- **ei ≤ Ti − Ei ∀i**;


- **di ≤ Li − Ti ∀i**.


# 2. Resolvendo usando o solver CBC

In [ ]:
!pip install pulp
!sudo apt-get install glpk-utils

In [31]:
import time
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpBinary, PULP_CBC_CMD, value

def ler_dados_instancia(caminho_arquivo):
    with open(caminho_arquivo, 'r') as arquivo:
        linha_inicial = arquivo.readline().strip().split()
        qtdeDeAvioes = int(linha_inicial[0])

        tempos = []
        separacoes = []
        penalidades = []

        for _ in range(qtdeDeAvioes):
            linha_tempos = arquivo.readline().strip().split()
            while len(linha_tempos) < 6:
                linha_tempos.extend(arquivo.readline().strip().split())
            ri, ei, ti, li, gi, hi = map(float, linha_tempos[:6])
            tempos.append((ri, ei, ti, li, gi, hi))
            penalidades.append((gi, hi))

            linha_separacao = []
            while len(linha_separacao) < qtdeDeAvioes:
                linha_separacao.extend(arquivo.readline().strip().split())
            separacoes.append(list(map(float, linha_separacao[:qtdeDeAvioes])))

        return qtdeDeAvioes, tempos, separacoes, penalidades


def resolver_modelo_glpk(qtdeDeAvioes, tempos, separacoes, penalidades):
    modelo = LpProblem("SequenciamentoPousos", LpMinimize)

    # Variáveis de decisão
    t = [LpVariable(f"t_{i}", lowBound=tempos[i][1], upBound=tempos[i][3], cat='Continuous') for i in range(qtdeDeAvioes)]
    e = [LpVariable(f"e_{i}", lowBound=0, cat='Continuous') for i in range(qtdeDeAvioes)]
    d = [LpVariable(f"d_{i}", lowBound=0, cat='Continuous') for i in range(qtdeDeAvioes)]
    x = [[LpVariable(f"x_{i}_{j}", cat=LpBinary) if i != j else None for j in range(qtdeDeAvioes)] for i in range(qtdeDeAvioes)]

    # Função objetivo
    modelo += lpSum(penalidades[i][0] * e[i] + penalidades[i][1] * d[i] for i in range(qtdeDeAvioes)), "PenalidadeTotal"

    # Constante grande M
    M = max(tempos[i][3] for i in range(qtdeDeAvioes)) + max(max(separacoes[i]) for i in range(qtdeDeAvioes))

    # Restrições de separação e sequenciamento
    for i in range(qtdeDeAvioes):
        for j in range(qtdeDeAvioes):
            if i != j:
                modelo += t[j] >= t[i] + separacoes[i][j] - M * (1 - x[i][j]), f"Separacao_{i}_{j}_A"
                modelo += t[i] >= t[j] + separacoes[j][i] - M * x[i][j], f"Separacao_{j}_{i}_B"
                modelo += x[i][j] + x[j][i] == 1, f"Sequencia_{i}_{j}"

    # Cálculo de adiantamento e atraso
    for i in range(qtdeDeAvioes):
        modelo += e[i] >= tempos[i][2] - t[i], f"Adiantamento_{i}"
        modelo += d[i] >= t[i] - tempos[i][2], f"Atraso_{i}"
        modelo += e[i] <= tempos[i][2] - tempos[i][1], f"LimiteAdiantamento_{i}"
        modelo += d[i] <= tempos[i][3] - tempos[i][2], f"LimiteAtraso_{i}"

    # Resolver o modelo com tempo limite
    inicio_tempo = time.time()
    resultado = modelo.solve(PULP_CBC_CMD(msg=True, timeLimit=640))
    tempo_execucao = time.time() - inicio_tempo

    if modelo.status == 1:  # Solução ótima encontrada
        tempos_pouso = [value(t[i]) for i in range(qtdeDeAvioes)]
        valor_otimo = value(modelo.objective)
        print(f"Valor ótimo da solução: {valor_otimo}")
        print(f"Tempos de pouso: {', '.join(map(str, tempos_pouso))}")
        print(f"Tempo de execução do Solver: {tempo_execucao:.2f} segundos")
    else:  # Solução aproximada
        tempos_pouso = [value(t[i]) for i in range(qtdeDeAvioes) if t[i].varValue is not None]
        valor_aproximado = value(modelo.objective) if value(modelo.objective) is not None else float('inf')
        print("Solução aproximada encontrada (não ótima dentro do tempo limite)")
        print(f"Penalidade aproximada: {valor_aproximado}")
        print(f"Tempos de pouso aproximados: {', '.join(map(str, tempos_pouso))}")
        print(f"Tempo de execução do Solver: {tempo_execucao:.2f} segundos")


def executarInstancia():
    for i in range(1, 9):
        caminho_arquivo = f"problema_do_aviao/instances/0{i}.dat"
        qtdeDeAvioes, tempos, separacoes, penalidades = ler_dados_instancia(caminho_arquivo)
        resolver_modelo_glpk(qtdeDeAvioes, tempos, separacoes, penalidades)


executarInstancia()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/guilherme/miniconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/1cd4fa8d0fe743a2b356e7654f345877-pulp.mps -sec 640 -timeMode elapsed -branch -printingOptions all -solution /tmp/1cd4fa8d0fe743a2b356e7654f345877-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 315 COLUMNS
At line 1296 RHS
At line 1607 BOUNDS
At line 1718 ENDATA
Problem MODEL has 310 rows, 120 columns and 780 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 640
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 9 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 7 strengthened rows, 0 substitutions
Cgl0004I processed model has 110 rows, 75 columns (45 integer (45 of which binary)) and 326 elements
Cbc0038I Initial state - 23 integers unsat

# 3. Resolvendo usando a Heurística (VNS)